# SVM и его ядра
__Суммарное количество баллов: 10__

В этом задании вам не нужно будет реализовать SVM, а научиться его применять.

Примените метод SVM (например, из библиотеки sklearn) для датасета blobs2.

Визуализируйте результат (разбиение плоскости и опорные вектора) при разных вариантах ядер (линейное; полиномиальное степеней 2,3,5; RBF).

In [ ]:
from sklearn.svm import SVC 
from sklearn.datasets import make_blobs
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

X_blobs2, y_blobs2 = make_blobs(n_samples=1000, random_state=4, center_box=(-2, 2),cluster_std=0.4, centers=2)

X_train, X_test, y_train, y_test = train_test_split(X_blobs2, y_blobs2)


In [ ]:
linear_svc = SVC(kernel='linear')
poly_2_svc = SVC(kernel='poly', degree=2)
poly_3_svc = SVC(kernel='poly', degree=3)
poly_5_svc = SVC(kernel='poly', degree=5)
rbf_svc = SVC(kernel='rbf')

svc_arr = [linear_svc, poly_2_svc, poly_3_svc, poly_5_svc, rbf_svc]

[svc.fit(X_train, y_train) for svc in svc_arr]

for svc in svc_arr:
  plt.figure(figsize=(12, 10))
  sns.scatterplot(
    data=pd.DataFrame([[*x, y] for x, y in zip(X_test, svc.predict(X_test))], columns=['x', 'y', 'hue']),
    x='x', y='y', hue='hue'
  )

# Get support vectors themselves
for svc in svc_arr:
  support_vectors = svc.support_vectors_
  
  # Visualize support vectors
  plt.figure(figsize=(12, 10))
  plt.scatter(X_train[:,0], X_train[:,1])
  plt.scatter(support_vectors[:,0], support_vectors[:,1])
  plt.show()

Протестируйте SVM на датасете spam. Для этой задачи используйте данные x_spam_train и y_spam_train:

1. Выполните загрузку и предобработку файлов x_spam_train и y_spam_train.
2. Разбейте x_spam_train и y_spam_train на x_train, y_train, x_test и y_test для оценки точности работы алгоритма.
3. Посчитайте метрики precision, recall, accuracy для модели SVM. Если необходимо, попробуйте разные наборы параметров для получения лучшего результата.
4. Сравните время работы SVM и Random Forest при близкой точности.

Ответьте на следующие вопросы:
1. Какой нужен препроцессинг данных для моделей?
2. Какая модель делает предсказания лучше? Предположите, почему.

1. Нужно отрезать колонку `id` т.к. она не является значимой для предсказания. Кроме того, данные для SVC необходимо отскалировать, чтобы адекватно оценивать расстояния
2. С дефолтными параметрами обе модели дают примерно одинаковый результат, но лес учится дольше. Предположительно потому что высчитывать оптимальные разбиения дерева и делить их несколько дольше, чем матричная математика

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.preprocessing import minmax_scale
from sklearn.ensemble import RandomForestClassifier
import time

X, y = pd.read_csv('./hw_trees_data/x_spam_train.csv', index_col='Id'), pd.read_csv('./hw_trees_data/y_spam_train.csv', index_col='Id')
X_scaled = minmax_scale(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

svc = SVC()
rfc = RandomForestClassifier()
start_time = time.time_ns()
svc.fit(X_train, y_train['Expected'])
svc_learning_time = time.time_ns() - start_time
start_time = time.time_ns()
rfc.fit(X_train, y_train['Expected'])
rfc_learning_time = time.time_ns() - start_time
svc_pred = svc.predict(X_test)
rfc_pred = svc.predict(X_test)

print(
  'SVC:',
  accuracy_score(y_test['Expected'], svc_pred),
  recall_score(y_test['Expected'], svc_pred),
  precision_score(y_test['Expected'], svc_pred),
  svc_learning_time / 1000
)

print(
  'RFC:',
  accuracy_score(y_test['Expected'], rfc_pred),
  recall_score(y_test['Expected'], rfc_pred),
  precision_score(y_test['Expected'], rfc_pred),
  rfc_learning_time / 1000
)

Обучите модель на всех данных из x_spam_train и y_spam_train.
Сделайте submit своего решения и получите значение f1_score не менее 0.6

In [ ]:
svc = SVC()
svc.fit(X_scaled, y)
test = pd.read_csv('hw_trees_data/x_spam_test.csv')
test_scaled = minmax_scale(test.drop('Id', axis=1))
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = test["Id"]
submission["Expected"] = svc.predict(test_scaled)
submission.to_csv('submission.csv', index=False)